<a href="https://colab.research.google.com/github/CodeAlamin2024/DSA210_PROJECT/blob/main/DataScience_Project_CRT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2_contingency, ttest_ind, pearsonr, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

plt.style.use("default")
sns.set_theme()

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 140)


LOAD DATASET

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!ls "/content/drive/MyDrive"

'10K Challenge Account.gsheet'
'20-day trading Challenge.gsheet'
 300_African_Flights.gsheet
 Advanced-Excel-Workout-Template.xlsx
'Airline Manager Stats.gsheet'
 Akbank_Receipt.pdf
'Autonomous Trading Signal and Execution System.gdoc'
'Avalanche(AVAX).gsheet'
 avax-usd-max.xls
'BackTest Account (AVAX).gsheet'
'BackTest Account (BTC).gsheet'
'BackTest Account (SOL).gsheet'
 Backtesting.gsheet
'BackTest of my SMC.gsheet'
'BackTest of SUI.gsheet'
'Bakctest document.gdoc'
"Between the Lines: A Trader's Journey to Inner Wealth.gdoc"
'Bitcoin(BTC).gsheet'
 bitcoin_price_data_cleaned.csv
'BTC Data Log Entries.gsheet'
'btc-usd-max (1).xls'
 btc-usd-max.xls
'Career Pilot X-Plane Log.gsheet'
'Cars to acquire based of Quarter.gdoc'
'Cleaning and Wrangling Data Using Spreadsheet.gdoc'
'Colab Notebooks'
'Copy of Impulse Candle Short Backtest.gsheet'
'Copy of Industrial Revolution Primary Source Docs.gdoc'
'Copy of Live Account.gsheet'
'Copy of Workout Tracker (1).gsheet'
'Copy of Workout Tracker.g

In [11]:
DATA_PATH = "/content/drive/MyDrive/GBPUSD CRT H4 - M15 BKTST.xlsx"
DATE_COL = "DATE"
SHEET_NAME = None

df_raw = pd.read_excel(DATA_PATH, sheet_name=0)
df_raw.head()


,GBPUSD 🇬🇧,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,DATE,ENTRY TIME,EXIT TIME,DURATION,PAIR,SESSION,STRATEGY,ENTRY TF,BIAS TF,ENTRY PRICE,STOP LOSS (SL),TAKE PROFIT (TP),POSITION (BUY/SELL),LOT SIZE,EXIT PRICE,PIPS GAINED / LOST,RISK-REWARD RATIO (RRR),WIN / LOSS,NOTES / MARKET CONTEXT,% RETURN
1,2025-01-03 00:00:00,16:15:00,14:00 (Next Day),21:45:00,GBPUSD,NEW YORK SESSION,CRT 4H - 15M,M15,4H,1.24026,1.23855,1.25055,BUY,0.05,1.25055,102.9,6.02,WIN,https://www.tradingview.com/x/tT1nb6zD/,6.02
2,2025-01-03 00:00:00,16:45:00,14:00 (Next Day),21:15:00,GBPUSD,NEW YORK SESSION,CRT 4H - 15M,M15,4H,1.23998,1.23855,1.25055,BUY,0.06,1.25055,105.8,7.35,WIN,https://www.tradingview.com/x/tT1nb6zD/,7.35
3,2025-01-10 00:00:00,11:00:00,16:30:00,05:30:00,GBPUSD,LONDON SESSION,CRT 4H - 15M,M15,4H,1.22846,1.22656,1.23246,BUY,0.05,1.22656,19,-1,LOSS,https://www.tradingview.com/x/lnDPWPmS/,-1
4,2025-01-13 00:00:00,16:45:00,01:30 (Next Day),08:45:00,GBPUSD,NEW YORK SESSION,CRT 4H - 15M,M15,4H,1.21383,1.20983,1.22181,BUY,0.02,1.22181,79.8,2,WIN,https://www.tradingview.com/x/08feEYlT/,2


DATA CLEANING

In [15]:
df = df_raw.copy()

# ---------------------------------------------------------
# 1. Rename columns correctly based on your actual headers
# ---------------------------------------------------------
rename_map = {
    "DATE ": "DATE",
    "ENTRY TIME": "ENTRY_TIME",
    "EXIT TIME": "EXIT_TIME",
    "DURATION": "DURATION",
    "PAIR": "PAIR",
    "SESSION": "SESSION",
    "STRATEGY": "STRATEGY",
    "ENTRY TF": "ENTRY_TF",
    "BIAS TF": "BIAS_TF",
    "ENTRY PRICE": "ENTRY_PRICE",
    "STOP LOSS (SL)": "STOP_LOSS",
    "TAKE PROFIT (TP)": "TAKE_PROFIT",
    "POSITION (BUY/SELL)": "POSITION",
    "LOT SIZE": "LOT_SIZE",
    "EXIT PRICE": "EXIT_PRICE",
    "PIPS GAINED / LOST": "PIPS",
    "RISK-REWARD RATIO (RRR)": "R",
    "WIN / LOSS": "WIN_LOSS",
    "NOTES / MARKET CONTEXT": "NOTES",
    "% RETURN": "RETURN_PCT",
}

df = df.rename(columns=rename_map)

# ---------------------------------------------------------
# 2. Parse DATE
# ---------------------------------------------------------
df["DATE"] = pd.to_datetime(df["DATE"])

# Day of week
df["weekday"] = df["DATE"].dt.day_name()

# ---------------------------------------------------------
# 3. Encode WIN/LOSS
# ---------------------------------------------------------
df["is_win"] = (df["WIN_LOSS"].str.upper().str.strip() == "WIN").astype(int)

# ---------------------------------------------------------
# 4. Normalize text columns
# ---------------------------------------------------------
df["POSITION"] = df["POSITION"].str.upper().str.strip()
df["SESSION"] = df["SESSION"].str.upper().str.strip()

# ---------------------------------------------------------
# 5. Convert DURATION "H:MM" → numeric hours
# ---------------------------------------------------------
def duration_to_hours(x):
    if pd.isna(x):
        return np.nan
    try:
        h, m = str(x).split(":")
        return int(h) + int(m)/60
    except:
        return np.nan

df["duration_hours"] = df["DURATION"].apply(duration_to_hours)

df.head()


,DATE,ENTRY_TIME,EXIT_TIME,DURATION,PAIR,SESSION,STRATEGY,ENTRY_TF,BIAS_TF,ENTRY_PRICE,STOP_LOSS,TAKE_PROFIT,POSITION,LOT_SIZE,EXIT_PRICE,PIPS,R,WIN_LOSS,NOTES,RETURN_PCT,weekday,is_win,duration_hours
0,2025-01-03,16:15:00,14:00 (Next Day),21:45:00,GBPUSD,NEW YORK SESSION,CRT 4H - 15M,M15,4H,1.24026,1.23855,1.25055,BUY,0.05,1.25055,102.9,6.02,WIN,https://www.tradingview.com/x/tT1nb6zD/,6.02,Friday,1,NaN
1,2025-01-03,16:45:00,14:00 (Next Day),21:15:00,GBPUSD,NEW YORK SESSION,CRT 4H - 15M,M15,4H,1.23998,1.23855,1.25055,BUY,0.06,1.25055,105.8,7.35,WIN,https://www.tradingview.com/x/tT1nb6zD/,7.35,Friday,1,NaN
2,2025-01-10,11:00:00,16:30:00,05:30:00,GBPUSD,LONDON SESSION,CRT 4H - 15M,M15,4H,1.22846,1.22656,1.23246,BUY,0.05,1.22656,19.0,-1.00,LOSS,https://www.tradingview.com/x/lnDPWPmS/,-1.00,Friday,0,NaN
3,2025-01-13,16:45:00,01:30 (Next Day),08:45:00,GBPUSD,NEW YORK SESSION,CRT 4H - 15M,M15,4H,1.21383,1.20983,1.22181,BUY,0.02,1.22181,79.8,2.00,WIN,https://www.tradingview.com/x/08feEYlT/,2.00,Monday,1,NaN
4,2025-01-14,07:00:00,12:15:00,05:15:00,GBPUSD,LONDON SESSION,CRT 4H - 15M,M15,4H,1.22075,1.21904,1.22503,BUY,0.05,1.21904,17.1,-1.00,LOSS,https://www.tradingview.com/x/qMOCEXvZ/,-1.00,Tuesday,0,NaN
